In [1]:
import logging
import io
from pathlib import Path
import gradio as gr
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import ChatMessage, ChatMessageRole
from maud.interface.config import load_config
import json
from PIL import Image

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Initialize the Databricks Workspace Client
workspace_client = WorkspaceClient()

# Load configuration
app_config = load_config("../implementations/interfaces/doc_retriever/app_config.yaml")

INFO:databricks.sdk:loading DEFAULT profile from ~/.databrickscfg: host, token


In [2]:
messages = [ChatMessage(role=ChatMessageRole.USER, content="What is machine learning?")]

In [3]:
response = workspace_client.serving_endpoints.query(
        name=app_config.serving_endpoint, messages=messages
    )

In [12]:
import requests

# Get the workspace URL and token from the workspace client
workspace_url = workspace_client.config.host
token = workspace_client.config.token

# Construct the serving endpoint URL


endpoint_url = f"{workspace_url}/serving-endpoints/{app_config.serving_endpoint}/invocations"

# Prepare the request headers
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

# Prepare the request payload
payload = {
    "messages": [
        {
            "role": "system", 
            "content": "Respond with So... and keep it short."
        },
        {
            "role": "user", 
            "content": "What is the factor of safety for strapping?"
        }
    ]
}

# Make the REST request
response = requests.post(
    endpoint_url,
    headers=headers,
    json=payload
)

# Check response status and get result
if response.status_code == 200:
    result = response.json()
    print(result)
else:
    print(f"Error: {response.status_code}")
    print(response.text)


{'choices': [{'message': {'role': 'assistant', 'content': "The factor of safety for fabric materials used in overhead lifts is usually 20, but it's recommended to consult the manufacturer's catalog for specific information on product characteristics and load ratings.", 'refusal': None, 'name': None, 'tool_calls': None, 'tool_call_id': None}, 'index': 0, 'finish_reason': 'stop', 'logprobs': None}], 'usage': None, 'id': 'b2308d5b-f801-44f0-9358-afdf079c8bf7', 'model': None, 'object': 'chat.completion', 'created': 1739334993, 'custom_outputs': {'message_history': [{'role': 'system', 'content': 'Respond with So... and keep it short.'}, {'role': 'user', 'content': 'What is the factor of safety for strapping?'}, {'role': 'tool', 'content': "Passage: A Fabric straps of woven material such as nylon, dacron, etc.are most often used for and hoisting the aircraft structure to keep the skin surfaces from being marred or scratched . Nylon and dacron webbings are more resistant to abrasion, have les

In [14]:
result['choices'][0]['message']['content']

"The factor of safety for fabric materials used in overhead lifts is usually 20, but it's recommended to consult the manufacturer's catalog for specific information on product characteristics and load ratings."

In [18]:
# Format documents for gradio chat display
documents = result.get("custom_outputs", {}).get("documents", [])
formatted_docs = []

for doc in documents:
    metadata = doc.get("metadata", {})
    content = doc.get("page_content", "")
    
    formatted_text = f"""
Source: {metadata.get('filename', 'Unknown')}
Type: {metadata.get('type', 'Unknown')}
ID: {metadata.get('id', 'Unknown')}

Content:
{content}
"""
    formatted_docs.append(formatted_text)

"\n\n---\n\n".join(formatted_docs)

"\nSource: D8200-ARM.pdf\nType: text\nID: 51539607558.0\n\nContent:\nA Fabric straps of woven material such as nylon, dacron, etc.are most often used for and hoisting the aircraft structure to keep the skin surfaces from being marred or scratched . Nylon and dacron webbings are more resistant to abrasion, have less moisture absorption, quick drying and are resistant to rot and mold. However , longed exposure to high temperature and low humidity will reduce its moisture content and consequently make it brittle and subject to failure It has been found that nylon straps exposed to 125 degrees F(52 degrees C.) air for 120 hours failed at lower than normal load, If these conditions are anticipated, polypropylene straps should be used instead of nylon. lifting pro-\nB In general, webbing straps must be matched closely to the service conditions and have adequate protective devices and increased factors of safety . Some webbing straps have wire woven in them for strength. Manufacturer's catalo